In [ ]:
%load_ext autoreload
%autoreload 2

import hydra
import os
import datetime
from pathlib import Path
from src.weighted_network import WeightedNetwork

# Initialize hydra and move to the root of the repository
try:
    hydra.initialize(version_base=None, config_path="../config/")
    CONFIG = hydra.compose(config_name="main.yaml")
    print('Initializing hydra')
except:
    print('Hydra already initalized!')
else:
    # Create an output folder in the root of the repository
    os.chdir('..')
    OUTPUT_FOLDER = Path('output/{0}'.format(datetime.datetime.now()))
    Path(OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

In [ ]:
def load(year0=None, year1=None, edge_filter='S238'):
    """Load the weighted network."""
    input_path = Path(CONFIG.data.raw)
    nodes_file = input_path / CONFIG.data.filenames.nodes
    edges_file = input_path / CONFIG.data.filenames.edges
    return WeightedNetwork(nodes_file=nodes_file, edges_file=edges_file, year0=year0, year1=year1, edge_filter=edge_filter)


In [ ]:
G_WEIGHTED = load()

nodes = G_WEIGHTED.nodes()
nodes = nodes[['PlaceID', 'PlaceName']].drop_duplicates()

In [ ]:
nodes = G_WEIGHTED.nodes()
nodes = nodes[['PlaceID', 'PlaceName']].drop_duplicates()

G_WEIGHTED.compute_communities(max_communities=10)
communities = G_WEIGHTED.communities()

for i, c in enumerate(communities):
    nodes.loc[nodes.PlaceID.isin(c), 'community10'] = i

G_WEIGHTED.compute_communities(max_communities=25)
communities = G_WEIGHTED.communities()

for i, c in enumerate(communities):
    nodes.loc[nodes.PlaceID.isin(c), 'community25'] = i

nodes.fillna(-1, inplace=True)
nodes.to_csv('computations/nodes_to_communities.csv', index=False)